<a href="https://colab.research.google.com/github/Sohaib1424/Data-Science-Kaggle/blob/main/notebooks/ft_deepsentipers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install accelerate datasets evaluate --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [2]:
model_name = "sbunlp/fabert"

In [3]:
from datasets import load_dataset, DatasetDict, Dataset

dsp = load_dataset("Khedesh/DeepSentiPers")
v_ = dsp.pop("validation")
dsp['test'] = v_
dsptr = dsp['train'].train_test_split(test_size = 0.1,seed = 42)
dsp['train'] = dsptr['train']
dsp['valid'] = dsptr['test']


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/807 [00:00<?, ?B/s]

(…)-00000-of-00001-1f4e901e128ad415.parquet:   0%|          | 0.00/596k [00:00<?, ?B/s]

(…)-00000-of-00001-a162f120fb654ea4.parquet:   0%|          | 0.00/194k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7023 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1854 [00:00<?, ? examples/s]

In [4]:
# convert from 5-label to ternary (-1,0,1)
lmap = {0:1,-1:0,-2:0,1:2,2:2}
dsp_ternary = DatasetDict({
    "train": Dataset.from_dict({
        "text": dsp["train"]["text"],
        "label": [lmap[label] for label in dsp["train"]["label"]],
    }),
    "test": Dataset.from_dict({
        "text": dsp["test"]["text"],
        "label": [lmap[label] for label in dsp["test"]["label"]],
    }),
    "valid": Dataset.from_dict({
        "text": dsp["valid"]["text"],
        "label": [lmap[label] for label in dsp["valid"]["label"]],
    })
})

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/18.3k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/552k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [6]:
def preprocess_function(examples):
    tokenized_text = tokenizer(examples['text'], truncation=True, padding="max_length", max_length=512)
    return tokenized_text

In [7]:
tokenized_dsp = dsp_ternary.map(preprocess_function, batched = True)

Map:   0%|          | 0/6320 [00:00<?, ? examples/s]

Map:   0%|          | 0/1854 [00:00<?, ? examples/s]

Map:   0%|          | 0/703 [00:00<?, ? examples/s]

In [8]:
import numpy as np
import evaluate
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy_result = accuracy.compute(predictions=predictions, references=labels)['accuracy']
    f1_result = f1.compute(predictions=predictions, references=labels,average='macro')['f1']
    return {"accuracy":accuracy_result,"f1":f1_result}


In [9]:
id2label = {
    0:"Negative",
    1:"Neutral",
    2:"Positive",
}
label2id = {
    "Negative":0,
    "Neutral":1,
    "Positive":2,
}

In [10]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [11]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(id2label),id2label=id2label,label2id=label2id)

config.json:   0%|          | 0.00/589 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sbunlp/fabert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
training_args = TrainingArguments(
    output_dir = "/tmp/dsp",
    learning_rate= 5e-5,
    per_device_train_batch_size= 16,
    per_device_eval_batch_size= 16,
    num_train_epochs= 3,
    weight_decay= 0,
    warmup_ratio = 0,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dsp["train"],
    eval_dataset=tokenized_dsp["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    )
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-12-73b57d7ce4ac>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sohaib-moradi4280 (sohaib-moradi4280-iran-university-of-science-and-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.603287,0.788051,0.732317
2,0.619400,0.577641,0.800853,0.754276
3,0.341300,0.605503,0.805121,0.764047


TrainOutput(global_step=1185, training_loss=0.4444186633146262, metrics={'train_runtime': 1892.5444, 'train_samples_per_second': 10.018, 'train_steps_per_second': 0.626, 'total_flos': 4988630400122880.0, 'train_loss': 0.4444186633146262, 'epoch': 3.0})

In [13]:
validation_results = trainer.evaluate()
print(validation_results)

{'eval_loss': 0.6055028438568115, 'eval_accuracy': 0.8051209103840683, 'eval_f1': 0.764047078920664, 'eval_runtime': 21.9722, 'eval_samples_per_second': 31.995, 'eval_steps_per_second': 2.003, 'epoch': 3.0}


In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dsp["train"],
    eval_dataset=tokenized_dsp["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    )
test_results = trainer.evaluate()
print(test_results)


<ipython-input-14-aef850a1b5df>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


{'eval_loss': 0.5528432726860046, 'eval_model_preparation_time': 0.0054, 'eval_accuracy': 0.8085221143473571, 'eval_f1': 0.7721607656934273, 'eval_runtime': 57.1117, 'eval_samples_per_second': 32.463, 'eval_steps_per_second': 2.031}


# Pipeline

In [15]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model = model,tokenizer = tokenizer,device=0)

Device set to use cuda:0


In [16]:
text = "از این خرید بسیار راضی هستم."
classifier(text)

[{'label': 'Positive', 'score': 0.9952534437179565}]

In [17]:
text = "پیشنهاد میکنم این محصول رو نگیرید"
classifier(text)

[{'label': 'Neutral', 'score': 0.8117930889129639}]